# Data Wrangling and Feature Engineering

## Preliminary steps

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import gc
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)

## Bureau Balance

### Description

1. Firstly, bureau balance dataset is preprocessed. Grouped on SK_ID_CURR we take aggregations for most important feature there MONTHS_BALANCE.
2. Bureau and bureau balance are merged on SK_ID_CURR
3. Extreme outliers are dropped, referencing EDA.
4. Various features engineering, such as credit duration, also various differences, or ratios.
5. Aggregations for continuous variables are computed.
6. Categorical variables are encoded to numerical representation.

### Processed Bureau Balance

In [2]:
class preprocess_bureau_and_bureau_balance:
    @staticmethod
    def preprocess_bureau_balance() -> pd.DataFrame:
        """
        Loads Bureau balance dataset and performs aggregations

        Returns:
            pd.DataFrame: Bureau balance with month's balance aggregations grouped on SK_ID_BUREAU
        """
        bureau_balance = pd.read_csv("../home-credit-default-risk/bureau_balance.csv")
        bureau_balance_aggregations = {"MONTHS_BALANCE": ["min", "max", "mean", "size"]}
        agg_bureau_balance = bureau_balance.groupby("SK_ID_BUREAU").agg(
            bureau_balance_aggregations
        )
        agg_bureau_balance.columns = pd.Index(
            [e[0] + "_" + e[1].upper() for e in agg_bureau_balance.columns.tolist()]
        )
        gc.collect()

        return agg_bureau_balance

    @staticmethod
    def preprocess_bureau(agg_bureau_balance: pd.DataFrame) -> pd.DataFrame:
        """
        Joins Bureau and Bureau balance datasets, then removes outlier referencing EDA.
        Engineers domain features, aggregates continuous variables, and converts categorical to dummies.

        Args:
            agg_bureau_balance (pd.DataFrame): Bureau Balance preprocessed dataframe

        Returns:
            pd.DataFrame: Bureau_n_balance of fully processed data grouped on SK_ID_CURR
        """
        bureau = pd.read_csv("../home-credit-default-risk/bureau.csv")
        bureau_n_balance = bureau.merge(
            agg_bureau_balance, how="left", on="SK_ID_BUREAU"
        )

        # Outliers
        bureau_n_balance.loc[
            bureau_n_balance["AMT_ANNUITY"] > 0.8e8, "AMT_ANNUITY"
        ] = np.nan
        bureau_n_balance.loc[
            bureau_n_balance["AMT_CREDIT_SUM"] > 3e8, "AMT_CREDIT_SUM"
        ] = np.nan
        bureau_n_balance.loc[
            bureau_n_balance["AMT_CREDIT_SUM_DEBT"] > 1e8, "AMT_CREDIT_SUM_DEBT"
        ] = np.nan
        bureau_n_balance.loc[
            bureau_n_balance["AMT_CREDIT_MAX_OVERDUE"] > 0.8e8, "AMT_CREDIT_MAX_OVERDUE"
        ] = np.nan
        bureau_n_balance.loc[
            bureau_n_balance["DAYS_ENDDATE_FACT"] < -10000, "DAYS_ENDDATE_FACT"
        ] = np.nan
        bureau_n_balance.loc[
            (bureau_n_balance["DAYS_CREDIT_UPDATE"] > 0)
            | (bureau["DAYS_CREDIT_UPDATE"] < -40000),
            "DAYS_CREDIT_UPDATE",
        ] = np.nan
        bureau_n_balance.loc[
            bureau_n_balance["DAYS_CREDIT_ENDDATE"] < -10000, "DAYS_CREDIT_ENDDATE"
        ] = np.nan
        bureau_n_balance.drop(
            bureau_n_balance[
                bureau_n_balance["DAYS_ENDDATE_FACT"] < bureau_n_balance["DAYS_CREDIT"]
            ].index,
            inplace=True,
        )

        # Domain feature engineering
        bureau_n_balance["CREDIT_DURATION"] = np.abs(
            bureau_n_balance["DAYS_CREDIT"] + bureau_n_balance["DAYS_CREDIT_ENDDATE"]
        )
        bureau_n_balance["FLAG_OVERDUE_RECENT"] = [
            0 if ele == 0 else 1 for ele in bureau_n_balance["CREDIT_DAY_OVERDUE"]
        ]
        bureau_n_balance["CNT_PROLONGED_MAX_OVERDUE_MUL"] = (
            bureau_n_balance["CNT_CREDIT_PROLONG"]
            * bureau_n_balance["AMT_CREDIT_MAX_OVERDUE"]
        )
        bureau_n_balance["CURRENT_AMT_OVERDUE_DURATION_RATIO"] = bureau_n_balance[
            "AMT_CREDIT_SUM_OVERDUE"
        ] / (bureau_n_balance["CREDIT_DURATION"] + 0.00001)
        bureau_n_balance["ENDDATE_DIF"] = (
            bureau_n_balance["DAYS_CREDIT_ENDDATE"]
            - bureau_n_balance["DAYS_ENDDATE_FACT"]
        )
        bureau_n_balance["DEBT_PERCENTAGE"] = bureau_n_balance["AMT_CREDIT_SUM"] / (
            bureau_n_balance["AMT_CREDIT_SUM_DEBT"] + 0.00001
        )
        bureau_n_balance["DEBT_CREDIT_DIFF"] = (
            bureau_n_balance["AMT_CREDIT_SUM"] - bureau_n_balance["AMT_CREDIT_SUM_DEBT"]
        )
        bureau_n_balance["CREDIT_TO_ANNUITY_RATIO"] = bureau_n_balance[
            "AMT_CREDIT_SUM"
        ] / (bureau_n_balance["AMT_ANNUITY"] + 0.00001)
        bureau_n_balance["ANNUITY_TO_CREDIT_RATIO"] = bureau_n_balance[
            "AMT_ANNUITY"
        ] / (bureau_n_balance["AMT_CREDIT_SUM"] + 0.00001)

        # Continuous
        categorical = bureau_n_balance.select_dtypes(
            exclude="number"
        ).columns.values.tolist()
        notcat = [
            x
            for x in bureau_n_balance.columns
            if x not in categorical + ["SK_ID_CURR", "SK_ID_BUREAU"]
        ]

        agg_bureau = bureau_n_balance.groupby("SK_ID_CURR").agg(
            {k: ["sum", "mean", "max", "min", "std"] for k in notcat}
        )
        agg_bureau.columns = pd.Index(
            ["bureau_" + e[0] + "_" + e[1] for e in agg_bureau.columns.tolist()]
        )

        # Categoricals
        dummy_temp = (
            pd.get_dummies(
                bureau_n_balance[["SK_ID_CURR"] + categorical], columns=categorical
            )
            .groupby("SK_ID_CURR")
            .sum()
        )
        for col in dummy_temp.columns:
            dummy_temp[col] = dummy_temp[col].astype(int)

        processed_bureau = pd.merge(
            agg_bureau,
            dummy_temp,
            left_on="SK_ID_CURR",
            right_on="SK_ID_CURR",
            how="left",
        )
        gc.collect()

        return processed_bureau

    def main(self) -> pd.DataFrame:
        aggregated_bureau_balance = self.preprocess_bureau_balance()
        bureau_merged_aggregated = self.preprocess_bureau(aggregated_bureau_balance)
        gc.collect()

        return bureau_merged_aggregated


In [3]:
bureau_aggregated = preprocess_bureau_and_bureau_balance().main()

In [4]:
bureau_aggregated.head()

,bureau_DAYS_CREDIT_sum,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_std,bureau_CREDIT_DAY_OVERDUE_sum,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,bureau_CREDIT_DAY_OVERDUE_std,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-5145,-735.000000,-49,-1572,489.942514,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
100002,-6992,-874.000000,-103,-1437,431.451040,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
100003,-5603,-1400.750000,-606,-2586,909.826128,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
100004,-1734,-867.000000,-408,-1326,649.124025,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
100005,-572,-190.666667,-62,-373,162.297053,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


## Pos cash balance

### Description

1. This dataset had no outliers removed here.
2. Only 2 features engineering: SK_DPD_RATIO and TOTAL_TERM.
3. Aggregations for the rest of the features grouped by SK_ID_CURR
4. Dummies for NAME_CONTRACT_STATUS

### Processed pos cash

In [5]:
class preprocess_pos_cash_balance:
    def __init__(self):
        pass

    def load_dataframe(self):
        self.pos_cash_balance = pd.read_csv(
            "../home-credit-default-risk/POS_CASH_balance.csv"
        )

    def load_and_feature_engineer(self):
        self.pos_cash_balance["SK_DPD_RATIO"] = self.pos_cash_balance["SK_DPD"] / (
            self.pos_cash_balance["SK_DPD_DEF"] + 0.00001
        )
        self.pos_cash_balance["TOTAL_TERM"] = (
            self.pos_cash_balance["CNT_INSTALMENT"]
            + self.pos_cash_balance["CNT_INSTALMENT_FUTURE"]
        )

    def aggregations(self) -> pd.DataFrame:
        agg_POS_CASH_balance = (
            self.pos_cash_balance.reset_index()
            .groupby("SK_ID_CURR")
            .agg(
                dict(
                    SK_DPD_RATIO=["mean", "max"],
                    TOTAL_TERM=["mean", "max", "last"],
                    MONTHS_BALANCE=["sum", "mean", "max", "min", "std"],
                    CNT_INSTALMENT=["sum", "mean", "max", "min", "std"],
                    CNT_INSTALMENT_FUTURE=["sum", "mean", "max", "min", "std"],
                    SK_DPD=["sum", "mean", "max", "min", "std"],
                    SK_DPD_DEF=["sum", "mean", "max", "min", "std"],
                    SK_ID_CURR="count",
                )
            )
        )

        agg_POS_CASH_balance.columns = pd.Index(
            [
                "PCASH_" + e[0] + "_" + e[1]
                for e in agg_POS_CASH_balance.columns.tolist()
            ]
        )

        return agg_POS_CASH_balance

    def get_dummies_and_merge(self, agg_POS_CASH_balance: pd.DataFrame) -> pd.DataFrame:

        contract_status = (
            pd.get_dummies(
                self.pos_cash_balance[["SK_ID_CURR", "NAME_CONTRACT_STATUS"]],
                columns=["NAME_CONTRACT_STATUS"],
            )
            .groupby("SK_ID_CURR")
            .sum()
        )
        for col in contract_status.columns:
            contract_status[col] = contract_status[col].astype(int)

        agg_POS_CASH_balance_merged = pd.merge(
            agg_POS_CASH_balance,
            contract_status,
            left_on="SK_ID_CURR",
            right_on="SK_ID_CURR",
            how="left",
        )

        return agg_POS_CASH_balance_merged

    def main(self) -> pd.DataFrame:
        self.load_dataframe()
        self.load_and_feature_engineer()
        aggregations = self.aggregations()
        pos_cash_balance = self.get_dummies_and_merge(aggregations)

        return pos_cash_balance


In [6]:
pos_aggregated = preprocess_pos_cash_balance().main()

In [7]:
pos_aggregated.head()

,PCASH_SK_DPD_RATIO_mean,PCASH_SK_DPD_RATIO_max,PCASH_TOTAL_TERM_mean,PCASH_TOTAL_TERM_max,PCASH_TOTAL_TERM_last,PCASH_MONTHS_BALANCE_sum,PCASH_MONTHS_BALANCE_mean,PCASH_MONTHS_BALANCE_max,PCASH_MONTHS_BALANCE_min,PCASH_MONTHS_BALANCE_std,...,PCASH_SK_ID_CURR_count,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.111111,0.999999,5.444444,8.0,7.0,-653,-72.555556,-53,-96,20.863312,...,9,7,0,0,0,2,0,0,0,0
100002,0.000000,0.000000,39.000000,48.0,44.0,-190,-10.000000,-1,-19,5.627314,...,19,19,0,0,0,0,0,0,0,0
100003,0.000000,0.000000,15.892857,24.0,12.0,-1226,-43.785714,-18,-77,24.640162,...,28,26,0,0,0,2,0,0,0,0
100004,0.000000,0.000000,6.000000,8.0,7.0,-102,-25.500000,-24,-27,1.290994,...,4,3,0,0,0,1,0,0,0,0
100005,0.000000,0.000000,18.900000,24.0,9.0,-220,-20.000000,-15,-25,3.316625,...,11,9,0,0,0,1,0,0,1,0


## Instalment payments

### Description

1. Only 2 criterias for outlier removal, referencing EDA.
2. Differences and ratios for feature engineering.
3. Aggregations for continuous variables grouped on SK_ID_CURR

### Processed instalments

In [8]:
class preprocess_installments_payments:
    def __init__(self):
        pass

    def load_dataframe(self):
        self.installments_payments = pd.read_csv(
            "../home-credit-default-risk/installments_payments.csv"
        )

    def replace_outliers(self):
        self.installments_payments.loc[
            self.installments_payments["NUM_INSTALMENT_VERSION"] > 100,
            "NUM_INSTALMENT_VERSION",
        ] = np.nan
        self.installments_payments.loc[
            self.installments_payments["DAYS_ENTRY_PAYMENT"] < -3500,
            "DAYS_ENTRY_PAYMENT",
        ] = np.nan

    def feature_engineering(self):
        self.installments_payments["DAYS_PAYMENT_RATIO"] = self.installments_payments[
            "DAYS_INSTALMENT"
        ] / (self.installments_payments["DAYS_ENTRY_PAYMENT"] + 0.00001)
        self.installments_payments["DAYS_PAYMENT_DIFF"] = (
            self.installments_payments["DAYS_INSTALMENT"]
            - self.installments_payments["DAYS_ENTRY_PAYMENT"]
        )
        self.installments_payments["AMT_PAYMENT_RATIO"] = self.installments_payments[
            "AMT_PAYMENT"
        ] / (self.installments_payments["AMT_INSTALMENT"] + 0.00001)
        self.installments_payments["AMT_PAYMENT_DIFF"] = (
            self.installments_payments["AMT_INSTALMENT"]
            - self.installments_payments["AMT_PAYMENT"]
        )

    def aggregations(self) -> pd.DataFrame:
        agg_installment_payments = (
            self.installments_payments.reset_index()
            .groupby("SK_ID_CURR")
            .agg(
                dict(
                    DAYS_PAYMENT_RATIO=["mean", "min", "max"],
                    DAYS_PAYMENT_DIFF=["mean", "min", "max"],
                    AMT_PAYMENT_RATIO=["mean", "min", "max"],
                    AMT_PAYMENT_DIFF=["mean", "min", "max"],
                    NUM_INSTALMENT_VERSION=["sum", "mean", "max", "min", "std"],
                    NUM_INSTALMENT_NUMBER=["sum", "mean", "max", "min", "std"],
                    DAYS_INSTALMENT=["sum", "mean", "max", "min", "std"],
                    DAYS_ENTRY_PAYMENT=["sum", "mean", "max", "min", "std"],
                    AMT_INSTALMENT=["sum", "mean", "max", "min", "std"],
                    AMT_PAYMENT=["sum", "mean", "max", "min", "std"],
                    SK_ID_CURR="count",
                )
            )
        )

        agg_installment_payments.columns = pd.Index(
            [
                "INSTALLMENT_" + e[0] + "_" + e[1]
                for e in agg_installment_payments.columns.tolist()
            ]
        )
        gc.collect()

        return agg_installment_payments

    def main(self) -> pd.DataFrame:
        self.load_dataframe()
        self.replace_outliers()
        self.feature_engineering()
        installments_payments = self.aggregations()
        gc.collect()

        return installments_payments

In [9]:
installments_payments = preprocess_installments_payments().main()

In [10]:
installments_payments.head()

,INSTALLMENT_DAYS_PAYMENT_RATIO_mean,INSTALLMENT_DAYS_PAYMENT_RATIO_min,INSTALLMENT_DAYS_PAYMENT_RATIO_max,INSTALLMENT_DAYS_PAYMENT_DIFF_mean,INSTALLMENT_DAYS_PAYMENT_DIFF_min,INSTALLMENT_DAYS_PAYMENT_DIFF_max,INSTALLMENT_AMT_PAYMENT_RATIO_mean,INSTALLMENT_AMT_PAYMENT_RATIO_min,INSTALLMENT_AMT_PAYMENT_RATIO_max,INSTALLMENT_AMT_PAYMENT_DIFF_mean,...,INSTALLMENT_AMT_INSTALMENT_mean,INSTALLMENT_AMT_INSTALMENT_max,INSTALLMENT_AMT_INSTALMENT_min,INSTALLMENT_AMT_INSTALMENT_std,INSTALLMENT_AMT_PAYMENT_sum,INSTALLMENT_AMT_PAYMENT_mean,INSTALLMENT_AMT_PAYMENT_max,INSTALLMENT_AMT_PAYMENT_min,INSTALLMENT_AMT_PAYMENT_std,INSTALLMENT_SK_ID_CURR_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.995312,0.979009,1.003826,7.285714,-11.0,36.0,1.0,1.0,1.0,0.0,...,5885.132143,17397.900,3951.000,5076.676624,41195.925,5885.132143,17397.900,3951.000,5076.676624,7
100002,0.901401,0.510204,0.962521,20.421053,12.0,31.0,1.0,1.0,1.0,0.0,...,11559.247105,53093.745,9251.775,10058.037722,219625.695,11559.247105,53093.745,9251.775,10058.037722,19
100003,0.992783,0.978820,0.999517,7.160000,1.0,14.0,1.0,1.0,1.0,0.0,...,64754.586000,560835.360,6662.970,110542.592300,1618864.650,64754.586000,560835.360,6662.970,110542.592300,25
100004,0.990080,0.986164,0.995873,7.666667,3.0,11.0,1.0,1.0,1.0,0.0,...,7096.155000,10573.965,5357.250,3011.871810,21288.465,7096.155000,10573.965,5357.250,3011.871810,3
100005,0.962653,0.945827,1.001709,23.555556,-1.0,37.0,1.0,1.0,1.0,0.0,...,6240.205000,17656.245,4813.200,4281.015000,56161.845,6240.205000,17656.245,4813.200,4281.015000,9


## Credit card balance

### Description

1. All outliers of AMT_PAYMENT_CURRENT above 4 millions were replaced as missing values.
2. Features developed based on sums, differences, ratios between variables.
3. Aggregations for continuous variables and dummy encoding for categorical. Both grouped on SK_ID_CURR.

### Processed credit card balance

In [11]:
class preprocess_credit_card_balance:
    def __init__(self):
        pass

    def load_dataframe(self):
        self.credit_card_balance = pd.read_csv(
            "../home-credit-default-risk/credit_card_balance.csv"
        )

    def replace_outliers(self):
        self.credit_card_balance.loc[
            self.credit_card_balance["AMT_PAYMENT_CURRENT"] > 4000000,
            "AMT_PAYMENT_CURRENT",
        ] = np.nan

    def feature_engineering(self):
        self.credit_card_balance["AMT_DRAWING_SUM"] = (
            self.credit_card_balance["AMT_DRAWINGS_ATM_CURRENT"]
            + self.credit_card_balance["AMT_DRAWINGS_CURRENT"]
            + self.credit_card_balance["AMT_DRAWINGS_OTHER_CURRENT"]
            + self.credit_card_balance["AMT_DRAWINGS_POS_CURRENT"]
        )
        self.credit_card_balance["BALANCE_LIMIT_RATIO"] = self.credit_card_balance[
            "AMT_BALANCE"
        ] / (self.credit_card_balance["AMT_CREDIT_LIMIT_ACTUAL"] + 0.00001)
        self.credit_card_balance["CNT_DRAWING_SUM"] = (
            self.credit_card_balance["CNT_DRAWINGS_ATM_CURRENT"]
            + self.credit_card_balance["CNT_DRAWINGS_CURRENT"]
            + self.credit_card_balance["CNT_DRAWINGS_OTHER_CURRENT"]
            + self.credit_card_balance["CNT_DRAWINGS_POS_CURRENT"]
            + self.credit_card_balance["CNT_INSTALMENT_MATURE_CUM"]
        )
        self.credit_card_balance["MIN_PAYMENT_RATIO"] = self.credit_card_balance[
            "AMT_PAYMENT_CURRENT"
        ] / (self.credit_card_balance["AMT_INST_MIN_REGULARITY"] + 0.0001)
        self.credit_card_balance["PAYMENT_MIN_DIFF"] = (
            self.credit_card_balance["AMT_PAYMENT_CURRENT"]
            - self.credit_card_balance["AMT_INST_MIN_REGULARITY"]
        )
        self.credit_card_balance["MIN_PAYMENT_TOTAL_RATIO"] = self.credit_card_balance[
            "AMT_PAYMENT_TOTAL_CURRENT"
        ] / (self.credit_card_balance["AMT_INST_MIN_REGULARITY"] + 0.00001)
        self.credit_card_balance["PAYMENT_MIN_DIFF"] = (
            self.credit_card_balance["AMT_PAYMENT_TOTAL_CURRENT"]
            - self.credit_card_balance["AMT_INST_MIN_REGULARITY"]
        )
        self.credit_card_balance["AMT_INTEREST_RECEIVABLE"] = (
            self.credit_card_balance["AMT_TOTAL_RECEIVABLE"]
            - self.credit_card_balance["AMT_RECEIVABLE_PRINCIPAL"]
        )
        self.credit_card_balance["SK_DPD_RATIO"] = self.credit_card_balance[
            "SK_DPD"
        ] / (self.credit_card_balance["SK_DPD_DEF"] + 0.00001)

    def aggregations_and_dummies(self) -> pd.DataFrame:

        categorical = self.credit_card_balance.select_dtypes(
            exclude="number"
        ).columns.values.tolist()
        notcat = [
            x
            for x in self.credit_card_balance.columns
            if x not in categorical + ["SK_ID_CURR", "SK_ID_PREV"]
        ]

        agg_credit = self.credit_card_balance.groupby("SK_ID_CURR").agg(
            {k: ["sum", "mean", "max", "min", "std"] for k in notcat}
        )
        agg_credit.columns = pd.Index(
            ["Credit_" + e[0] + "_" + e[1] for e in agg_credit.columns.tolist()]
        )

        contract_status = (
            pd.get_dummies(
                self.credit_card_balance[["SK_ID_CURR", "NAME_CONTRACT_STATUS"]],
                columns=["NAME_CONTRACT_STATUS"],
            )
            .groupby("SK_ID_CURR")
            .sum()
        )
        for col in contract_status.columns:
            contract_status[col] = contract_status[col].astype(int)

        agg_credit_balance = pd.merge(
            agg_credit,
            contract_status,
            left_on="SK_ID_CURR",
            right_on="SK_ID_CURR",
            how="left",
        )

        return agg_credit_balance

    def main(self) -> pd.DataFrame:
        self.load_dataframe()
        self.replace_outliers()
        self.feature_engineering()
        agg_credit_balance = self.aggregations_and_dummies()

        return agg_credit_balance


In [12]:
credit_card_balance = preprocess_credit_card_balance().main()

In [13]:
credit_card_balance.head()

,Credit_MONTHS_BALANCE_sum,Credit_MONTHS_BALANCE_mean,Credit_MONTHS_BALANCE_max,Credit_MONTHS_BALANCE_min,Credit_MONTHS_BALANCE_std,Credit_AMT_BALANCE_sum,Credit_AMT_BALANCE_mean,Credit_AMT_BALANCE_max,Credit_AMT_BALANCE_min,Credit_AMT_BALANCE_std,...,Credit_SK_DPD_RATIO_max,Credit_SK_DPD_RATIO_min,Credit_SK_DPD_RATIO_std,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-21,-3.5,-1,-6,1.870829,0.000,0.000000,0.00,0.0,0.000000,...,0.00000,0.0,0.000000,6,0,0,0,0,0,0
100011,-2849,-38.5,-2,-75,21.505813,4031676.225,54482.111149,189000.00,0.0,68127.238270,...,0.00000,0.0,0.000000,74,0,0,0,0,0,0
100013,-4656,-48.5,-1,-96,27.856777,1743352.245,18159.919219,161420.22,0.0,43237.406997,...,0.99999,0.0,0.102061,96,0,0,0,0,0,0
100021,-170,-10.0,-2,-18,5.049752,0.000,0.000000,0.00,0.0,0.000000,...,0.00000,0.0,0.000000,7,0,10,0,0,0,0
100023,-60,-7.5,-4,-11,2.449490,0.000,0.000000,0.00,0.0,0.000000,...,0.00000,0.0,0.000000,8,0,0,0,0,0,0


## Previous application

### Description

1. Replacing unusual outliers from EDA as missing values.
2. A lot of domain feature engineering with various variables.
3. Aggregations for continuous variables grouped on SK_ID_CURR

### Processed previous application

In [14]:
class preprocess_previous_application:
    def __init__(self):
        pass

    def load_dataframe(self):
        self.previous_application = pd.read_csv(
            "../home-credit-default-risk/previous_application.csv"
        )

    def replace_outliers(self):
        self.previous_application["AMT_DOWN_PAYMENT"] = np.where(
            self.previous_application["AMT_DOWN_PAYMENT"] < 0,
            0,
            self.previous_application["AMT_DOWN_PAYMENT"],
        )
        self.previous_application["DAYS_FIRST_DRAWING"].replace(
            365243, np.nan, inplace=True
        )
        self.previous_application["DAYS_FIRST_DUE"].replace(
            365243, np.nan, inplace=True
        )
        self.previous_application["DAYS_LAST_DUE_1ST_VERSION"].replace(
            365243, np.nan, inplace=True
        )
        self.previous_application["DAYS_LAST_DUE"].replace(365243, np.nan, inplace=True)
        self.previous_application["DAYS_TERMINATION"].replace(
            365243, np.nan, inplace=True
        )
        self.previous_application["SELLERPLACE_AREA"].replace(
            4000000, np.nan, inplace=True
        )

    def feature_engineering(self):
        self.previous_application[
            "MISSING_VALUES_TOTAL_PREV"
        ] = self.previous_application.isna().sum(axis=1)
        self.previous_application["AMT_DECLINED"] = (
            self.previous_application["AMT_APPLICATION"]
            - self.previous_application["AMT_CREDIT"]
        )
        self.previous_application["AMT_CREDIT_GOODS_RATIO"] = self.previous_application[
            "AMT_CREDIT"
        ] / (self.previous_application["AMT_GOODS_PRICE"] + 0.00001)
        self.previous_application["AMT_CREDIT_GOODS_DIFF"] = (
            self.previous_application["AMT_CREDIT"]
            - self.previous_application["AMT_GOODS_PRICE"]
        )
        self.previous_application[
            "AMT_CREDIT_APPLICATION_RATIO"
        ] = self.previous_application["AMT_APPLICATION"] / (
            self.previous_application["AMT_CREDIT"] + 0.00001
        )
        self.previous_application[
            "CREDIT_DOWNPAYMENT_RATIO"
        ] = self.previous_application["AMT_DOWN_PAYMENT"] / (
            self.previous_application["AMT_CREDIT"] + 0.00001
        )
        self.previous_application["GOOD_DOWNPAYMET_RATIO"] = self.previous_application[
            "AMT_DOWN_PAYMENT"
        ] / (self.previous_application["AMT_GOODS_PRICE"] + 0.00001)
        self.previous_application["INTEREST_DOWNPAYMENT"] = (
            self.previous_application["RATE_DOWN_PAYMENT"]
            * self.previous_application["AMT_DOWN_PAYMENT"]
        )
        self.previous_application["INTEREST_CREDIT"] = (
            self.previous_application["AMT_CREDIT"]
            * self.previous_application["RATE_INTEREST_PRIMARY"]
        )
        self.previous_application["INTEREST_CREDIT_PRIVILEGED"] = (
            self.previous_application["AMT_CREDIT"]
            * self.previous_application["RATE_INTEREST_PRIVILEGED"]
        )
        self.previous_application["APPLICATION_AMT_TO_DECISION_RATIO"] = (
            self.previous_application["AMT_APPLICATION"]
            / (self.previous_application["DAYS_DECISION"] + 0.00001)
            * -1
        )
        self.previous_application[
            "AMT_APPLICATION_TO_SELLERPLACE_AREA"
        ] = self.previous_application["AMT_APPLICATION"] / (
            self.previous_application["SELLERPLACE_AREA"] + 0.00001
        )
        self.previous_application["ANNUITY"] = self.previous_application[
            "AMT_CREDIT"
        ] / (self.previous_application["CNT_PAYMENT"] + 0.00001)
        self.previous_application["ANNUITY_GOODS"] = self.previous_application[
            "AMT_GOODS_PRICE"
        ] / (self.previous_application["CNT_PAYMENT"] + 0.00001)
        self.previous_application["DAYS_FIRST_LAST_DUE_DIFF"] = (
            self.previous_application["DAYS_LAST_DUE"]
            - self.previous_application["DAYS_FIRST_DUE"]
        )
        self.previous_application["AMT_CREDIT_HOUR_PROCESS_START"] = (
            self.previous_application["AMT_CREDIT"]
            * self.previous_application["HOUR_APPR_PROCESS_START"]
        )
        self.previous_application["AMT_CREDIT_NFLAG_LAST_APPL_DAY"] = (
            self.previous_application["AMT_CREDIT"]
            * self.previous_application["NFLAG_LAST_APPL_IN_DAY"]
        )
        self.previous_application["AMT_INTEREST"] = (
            self.previous_application["CNT_PAYMENT"]
            * self.previous_application["AMT_ANNUITY"]
            - self.previous_application["AMT_CREDIT"]
        )
        self.previous_application["INTEREST_SHARE"] = self.previous_application[
            "AMT_INTEREST"
        ] / (self.previous_application["AMT_CREDIT"] + 0.00001)
        self.previous_application["INTEREST_RATE"] = (
            2
            * 12
            * self.previous_application["AMT_INTEREST"]
            / (
                self.previous_application["AMT_CREDIT"]
                * (self.previous_application["CNT_PAYMENT"] + 1)
            )
        )

    def categorical_to_dummies(self) -> pd.DataFrame:
        continuous_var = [
            x
            for x in self.previous_application.select_dtypes(include="number").columns
            if x
            not in [
                "SK_ID_PREV",
                "SK_ID_CURR",
                "SELLERPLACE_AREA",
                "NFLAG_LAST_APPL_IN_DAY",
                "NFLAG_INSURED_ON_APPROVAL",
            ]
        ]
        categorical_var = [
            x
            for x in self.previous_application.columns
            if x not in continuous_var + ["SK_ID_CURR", "SK_ID_PREV"]
        ]
        important_categorical = [
            "FLAG_LAST_APPL_PER_CONTRACT",
            "NAME_CONTRACT_STATUS",
            "NAME_CLIENT_TYPE",
            "NAME_PORTFOLIO",
            "NAME_YIELD_GROUP",
        ]

        contract_status = (
            pd.get_dummies(
                self.previous_application[["SK_ID_CURR"] + important_categorical],
                columns=important_categorical,
            )
            .groupby("SK_ID_CURR")
            .sum()
        )
        for col in contract_status.columns:
            contract_status[col] = contract_status[col].astype(int)

        agg_previous_application = self.previous_application.groupby("SK_ID_CURR").agg(
            {k: ["nunique"] for k in categorical_var}
        )
        agg_previous_application.columns = pd.Index(
            [
                "PREV2_" + e[0] + "_" + e[1]
                for e in agg_previous_application.columns.tolist()
            ]
        )

        categorical_grouped = pd.merge(
            contract_status,
            agg_previous_application,
            left_on="SK_ID_CURR",
            right_on="SK_ID_CURR",
            how="left",
        )

        return categorical_grouped

    def aggregations(self, categorical_grouped) -> pd.DataFrame:
        continuous_var = [
            x
            for x in self.previous_application.select_dtypes(include="number").columns
            if x
            not in [
                "SK_ID_PREV",
                "SK_ID_CURR",
                "SELLERPLACE_AREA",
                "NFLAG_LAST_APPL_IN_DAY",
                "NFLAG_INSURED_ON_APPROVAL",
            ]
        ]

        approved_agg = (
            self.previous_application[
                self.previous_application["NAME_CONTRACT_STATUS"] == "Approved"
            ]
            .groupby("SK_ID_CURR")
            .agg({k: ["sum", "mean", "max", "min", "std"] for k in continuous_var})
        )
        approved_agg.columns = pd.Index(
            [
                "APPROVED_" + e[0] + "_" + e[1].upper()
                for e in approved_agg.columns.tolist()
            ]
        )

        refused_agg = (
            self.previous_application[
                self.previous_application["NAME_CONTRACT_STATUS"] == "Refused"
            ]
            .groupby("SK_ID_CURR")
            .agg({k: ["sum", "mean", "max", "min", "std"] for k in continuous_var})
        )
        refused_agg.columns = pd.Index(
            [
                "REFUSED_" + e[0] + "_" + e[1].upper()
                for e in refused_agg.columns.tolist()
            ]
        )

        approved_and_refused = pd.merge(
            approved_agg,
            refused_agg,
            left_on="SK_ID_CURR",
            right_on="SK_ID_CURR",
            how="left",
        )
        agg_previous_application = pd.merge(
            approved_and_refused,
            categorical_grouped,
            left_on="SK_ID_CURR",
            right_on="SK_ID_CURR",
            how="left",
        )

        return agg_previous_application

    def main(self) -> pd.DataFrame:
        self.load_dataframe()
        self.replace_outliers()
        self.feature_engineering()
        catogrical_to_dummies = self.categorical_to_dummies()
        agg_previous_application = self.aggregations(catogrical_to_dummies)

        return agg_previous_application


In [15]:
agg_previous_application = preprocess_previous_application().main()

In [16]:
agg_previous_application.head()

,APPROVED_AMT_ANNUITY_SUM,APPROVED_AMT_ANNUITY_MEAN,APPROVED_AMT_ANNUITY_MAX,APPROVED_AMT_ANNUITY_MIN,APPROVED_AMT_ANNUITY_STD,APPROVED_AMT_APPLICATION_SUM,APPROVED_AMT_APPLICATION_MEAN,APPROVED_AMT_APPLICATION_MAX,APPROVED_AMT_APPLICATION_MIN,APPROVED_AMT_APPLICATION_STD,...,PREV2_NAME_CLIENT_TYPE_nunique,PREV2_NAME_GOODS_CATEGORY_nunique,PREV2_NAME_PORTFOLIO_nunique,PREV2_NAME_PRODUCT_TYPE_nunique,PREV2_CHANNEL_TYPE_nunique,PREV2_SELLERPLACE_AREA_nunique,PREV2_NAME_SELLER_INDUSTRY_nunique,PREV2_NAME_YIELD_GROUP_nunique,PREV2_PRODUCT_COMBINATION_nunique,PREV2_NFLAG_INSURED_ON_APPROVAL_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,3951.000,NaN,24835.5,24835.5,24835.5,24835.5,NaN,...,1,1,1,1,1,1,1,1,1,1
100002,9251.775,9251.775,9251.775,9251.775,NaN,179055.0,179055.0,179055.0,179055.0,NaN,...,1,1,1,1,1,1,1,1,1,1
100003,169661.970,56553.990,98356.995,6737.310,46332.557777,1306309.5,435436.5,900000.0,68809.5,424161.620549,...,2,3,2,2,3,3,3,2,3,2
100004,5357.250,5357.250,5357.250,5357.250,NaN,24282.0,24282.0,24282.0,24282.0,NaN,...,1,1,1,1,1,1,1,1,1,1
100005,4813.200,4813.200,4813.200,4813.200,NaN,44617.5,44617.5,44617.5,44617.5,NaN,...,2,2,2,1,2,2,2,2,2,1


## Application

### Description

1. Removed some FLAG_DOCUMENT columns with only 1 value.
2. Replaced 'XNA' for gender variable with missing values.
3. Replaced unusual value of 365243 as missing value in time series.
4. Reduced cardinality for ORGANIZATION_TYPE
5. Feature engineering for the following categories of variables

* Income and credit features
* Age ratios and differences
* Car ratios
* Flagged contracts
* Family members
* Region ratings
* Flag regions
* External source
* Apartment scores
* OBS And DEF
* Flag Documents
* Details change
* Enquiries
* Binary

6. Imputed the mean of 500 nearest neighbor's target values for each application. Neighbors are computed using EXT_SOURCE feature and CREDIT_ANNUITY_RATIO.
7. Multiple aggregations based on various categorical variable groupings.

### Processed application

In [17]:
class preprocess_application_train_test:

    def __init__(self):
        pass


    def load_dataframe(self):
        self.application_train = pd.read_csv('../home-credit-default-risk/application_train.csv')
        self.application_test = pd.read_csv('../home-credit-default-risk/application_test.csv')


    def clean_dataset(self):
        """
        Clean application dataset by dropping outliers, replacing unusual values, reducing cardinality.
        """
        flag_cols_to_drop = ['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12',
                            'FLAG_DOCUMENT_20']
        self.application_train = self.application_train.drop(flag_cols_to_drop, axis = 1)
        self.application_test = self.application_test.drop(flag_cols_to_drop, axis = 1)

        self.application_train['CODE_GENDER'].replace('XNA', np.nan, inplace=True)
        self.application_test['CODE_GENDER'].replace('XNA', np.nan, inplace=True)

        self.application_train["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)
        self.application_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace=True)

        self.application_train.loc[self.application_train['OWN_CAR_AGE'] > 80, 'OWN_CAR_AGE'] = np.nan
        self.application_test.loc[self.application_test['OWN_CAR_AGE'] > 80, 'OWN_CAR_AGE'] = np.nan

        self.application_train['ORGANIZATION_TYPE'].mask(
            self.application_train['ORGANIZATION_TYPE'].map(
                self.application_train['ORGANIZATION_TYPE'].value_counts(normalize=True)) < 0.02, 'Other_2', inplace=True)

        self.application_test['ORGANIZATION_TYPE'].mask(
            self.application_test['ORGANIZATION_TYPE'].map(
                self.application_test['ORGANIZATION_TYPE'].value_counts(normalize=True)) < 0.02, 'Other_2', inplace=True)

    
    def feature_engineering(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Multiple newly engineering domain based features.

        Args:
            df (pd.DataFrame): train or test dataset for feature engineering

        Returns:
            pd.DataFrame: Inputed dataset returned with new features included
        """
        # Income and credit features
        df['CREDIT_INCOME_RATIO'] = df['AMT_CREDIT'] / (df['AMT_INCOME_TOTAL'] + 0.00001)
        df['CREDIT_ANNUITY_RATIO'] = df['AMT_CREDIT'] / (df['AMT_ANNUITY'] + 0.00001)
        df['ANNUITY_INCOME_RATIO'] = df['AMT_ANNUITY'] / (df['AMT_INCOME_TOTAL'] + 0.00001)
        df['INCOME_ANNUITY_DIFF'] = df['AMT_INCOME_TOTAL'] - df['AMT_ANNUITY']
        df['CREDIT_GOODS_RATIO'] = df['AMT_CREDIT'] / (df['AMT_GOODS_PRICE'] + 0.00001)
        df['CREDIT_GOODS_DIFF'] = df['AMT_CREDIT'] - df['AMT_GOODS_PRICE'] + 0.00001
        df['GOODS_INCOME_RATIO'] = df['AMT_GOODS_PRICE'] / (df['AMT_INCOME_TOTAL'] + 0.00001)
        df['INCOME_EXT_RATIO'] = df['AMT_INCOME_TOTAL'] / (df['EXT_SOURCE_3'] + 0.00001)
        df['CREDIT_EXT_RATIO'] = df['AMT_CREDIT'] / (df['EXT_SOURCE_3'] + 0.00001)
        # Age ratios and diffs
        df['AGE_EMPLOYED_DIFF'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
        df['EMPLOYED_TO_AGE_RATIO'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 0.00001)
        # Car ratios
        df['CAR_EMPLOYED_DIFF'] = df['OWN_CAR_AGE'] - df['DAYS_EMPLOYED']
        df['CAR_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / (df['DAYS_EMPLOYED']+0.00001)
        df['CAR_AGE_DIFF'] = df['DAYS_BIRTH'] - df['OWN_CAR_AGE']
        df['CAR_AGE_RATIO'] = df['OWN_CAR_AGE'] / (df['DAYS_BIRTH'] + 0.00001)
        # Flag contacts sum
        df['FLAG_CONTACTS_SUM'] = df['FLAG_MOBIL'] + df['FLAG_EMP_PHONE'] + df['FLAG_WORK_PHONE'] + df[
                                    'FLAG_CONT_MOBILE'] + df['FLAG_PHONE'] + df['FLAG_EMAIL']
        
        df['HOUR_PROCESS_CREDIT_MUL'] = df['AMT_CREDIT'] * df['HOUR_APPR_PROCESS_START']
        # Family members
        df['CNT_NON_CHILDREN'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
        df['CHILDREN_INCOME_RATIO'] = df['CNT_CHILDREN'] / (df['AMT_INCOME_TOTAL'] + 0.00001)
        df['PER_CAPITA_INCOME'] = df['AMT_INCOME_TOTAL'] / (df['CNT_FAM_MEMBERS'] + 1)
        # Region ratings
        df['REGIONS_RATING_INCOME_MUL'] = (df['REGION_RATING_CLIENT'] + df['REGION_RATING_CLIENT_W_CITY']) * df['AMT_INCOME_TOTAL'] / 2
        df['REGION_RATING_MAX'] = [max(ele1, ele2) for ele1, ele2 in zip(df['REGION_RATING_CLIENT'], df['REGION_RATING_CLIENT_W_CITY'])]
        df['REGION_RATING_MAX'] = [min(ele1, ele2) for ele1, ele2 in zip(df['REGION_RATING_CLIENT'], df['REGION_RATING_CLIENT_W_CITY'])]
        df['REGION_RATING_MEAN'] = (df['REGION_RATING_CLIENT'] + df['REGION_RATING_CLIENT_W_CITY']) / 2
        df['REGION_RATING_MUL'] = df['REGION_RATING_CLIENT'] * df['REGION_RATING_CLIENT_W_CITY']
        # Flag regions
        df['FLAG_REGIONS'] = df['REG_REGION_NOT_LIVE_REGION'] + df['REG_REGION_NOT_WORK_REGION'] + df['LIVE_REGION_NOT_WORK_REGION']+df[
                                'REG_CITY_NOT_LIVE_CITY'] + df['REG_CITY_NOT_WORK_CITY'] + df['LIVE_CITY_NOT_WORK_CITY']   
        # Ext_sources
        df['EXT_SOURCE_MEAN'] = (df['EXT_SOURCE_1'] + df['EXT_SOURCE_2'] + df['EXT_SOURCE_3'] ) / 3
        df['EXT_SOURCE_MUL'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3'] 
        df['EXT_SOURCE_MAX'] = [max(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(df['EXT_SOURCE_1'], df['EXT_SOURCE_2'], df['EXT_SOURCE_3'])]
        df['EXT_SOURCE_MIN'] = [min(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(df['EXT_SOURCE_1'], df['EXT_SOURCE_2'], df['EXT_SOURCE_3'])]
        df['EXT_SOURCE_VAR'] = [np.var([ele1,ele2,ele3]) for ele1, ele2, ele3 in zip(df['EXT_SOURCE_1'], df['EXT_SOURCE_2'], df['EXT_SOURCE_3'])]
        df['WEIGHTED_EXT_SOURCE'] =  df["EXT_SOURCE_1"] * 2 + df["EXT_SOURCE_2"] * 3 + df["EXT_SOURCE_3"] * 4
        # Apartment scores
        df['APARTMENTS_SUM_AVG'] = df['APARTMENTS_AVG'] + df['BASEMENTAREA_AVG'] + df['YEARS_BEGINEXPLUATATION_AVG'] + df[
                                    'YEARS_BUILD_AVG'] + df['COMMONAREA_AVG'] + df['ELEVATORS_AVG'] + df['ENTRANCES_AVG'] + df[
                                    'FLOORSMAX_AVG'] + df['FLOORSMIN_AVG'] + df['LANDAREA_AVG'] + df['LIVINGAPARTMENTS_AVG'] + df[
                                    'LIVINGAREA_AVG'] + df['NONLIVINGAPARTMENTS_AVG'] + df['NONLIVINGAREA_AVG']

        df['APARTMENTS_SUM_MODE'] = df['APARTMENTS_MODE'] + df['BASEMENTAREA_MODE'] + df['YEARS_BEGINEXPLUATATION_MODE'] + df[
                                    'YEARS_BUILD_MODE'] + df['COMMONAREA_MODE'] + df['ELEVATORS_MODE'] + df['ENTRANCES_MODE'] + df[
                                    'FLOORSMAX_MODE'] + df['FLOORSMIN_MODE'] + df['LANDAREA_MODE'] + df['LIVINGAPARTMENTS_MODE'] + df[
                                    'LIVINGAREA_MODE'] + df['NONLIVINGAPARTMENTS_MODE'] + df['NONLIVINGAREA_MODE'] + df['TOTALAREA_MODE']

        df['APARTMENTS_SUM_MEDI'] = df['APARTMENTS_MEDI'] + df['BASEMENTAREA_MEDI'] + df['YEARS_BEGINEXPLUATATION_MEDI'] + df[
                                    'YEARS_BUILD_MEDI'] + df['COMMONAREA_MEDI'] + df['ELEVATORS_MEDI'] + df['ENTRANCES_MEDI'] + df[
                                    'FLOORSMAX_MEDI'] + df['FLOORSMIN_MEDI'] + df['LANDAREA_MEDI'] + df['LIVINGAPARTMENTS_MEDI'] + df[
                                    'LIVINGAREA_MEDI'] + df['NONLIVINGAPARTMENTS_MEDI'] + df['NONLIVINGAREA_MEDI']
        df['INCOME_APARTMENT_AVG_MUL'] = df['APARTMENTS_SUM_AVG'] * df['AMT_INCOME_TOTAL']
        df['INCOME_APARTMENT_MODE_MUL'] = df['APARTMENTS_SUM_MODE'] * df['AMT_INCOME_TOTAL']
        df['INCOME_APARTMENT_MEDI_MUL'] = df['APARTMENTS_SUM_MEDI'] * df['AMT_INCOME_TOTAL']
        # OBS And DEF
        df['OBS_30_60_SUM'] = df['OBS_30_CNT_SOCIAL_CIRCLE'] + df['OBS_60_CNT_SOCIAL_CIRCLE']
        df['DEF_30_60_SUM'] = df['DEF_30_CNT_SOCIAL_CIRCLE'] + df['DEF_60_CNT_SOCIAL_CIRCLE']
        df['OBS_DEF_30_MUL'] = df['OBS_30_CNT_SOCIAL_CIRCLE'] *  df['DEF_30_CNT_SOCIAL_CIRCLE']
        df['OBS_DEF_60_MUL'] = df['OBS_60_CNT_SOCIAL_CIRCLE'] *  df['DEF_60_CNT_SOCIAL_CIRCLE']
        df['SUM_OBS_DEF_ALL'] = df['OBS_30_CNT_SOCIAL_CIRCLE'] + df['DEF_30_CNT_SOCIAL_CIRCLE'] + df[
                                    'OBS_60_CNT_SOCIAL_CIRCLE'] + df['DEF_60_CNT_SOCIAL_CIRCLE']
        df['OBS_30_CREDIT_RATIO'] = df['AMT_CREDIT'] / (df['OBS_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
        df['OBS_60_CREDIT_RATIO'] = df['AMT_CREDIT'] / (df['OBS_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
        df['DEF_30_CREDIT_RATIO'] = df['AMT_CREDIT'] / (df['DEF_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
        df['DEF_60_CREDIT_RATIO'] = df['AMT_CREDIT'] / (df['DEF_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
        # Flag Documents combined
        df['SUM_FLAGS_DOCUMENTS'] = df['FLAG_DOCUMENT_3'] + df['FLAG_DOCUMENT_5'] + df['FLAG_DOCUMENT_6']  + df[
                                    'FLAG_DOCUMENT_7'] + df['FLAG_DOCUMENT_8'] + df['FLAG_DOCUMENT_9'] + df[
                                    'FLAG_DOCUMENT_11'] + df['FLAG_DOCUMENT_13'] + df['FLAG_DOCUMENT_14'] + df[
                                    'FLAG_DOCUMENT_15'] + df['FLAG_DOCUMENT_16'] + df['FLAG_DOCUMENT_17'] + df[
                                    'FLAG_DOCUMENT_18'] + df['FLAG_DOCUMENT_19'] + df['FLAG_DOCUMENT_21']
        # Details change
        df['DAYS_DETAILS_CHANGE_MUL'] = df['DAYS_LAST_PHONE_CHANGE'] * df['DAYS_REGISTRATION'] * df['DAYS_ID_PUBLISH']
        df['DAYS_DETAILS_CHANGE_SUM'] = df['DAYS_LAST_PHONE_CHANGE'] + df['DAYS_REGISTRATION'] + df['DAYS_ID_PUBLISH']
        # Enquires
        df['AMT_ENQ_SUM'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'] + df['AMT_REQ_CREDIT_BUREAU_DAY'] + df['AMT_REQ_CREDIT_BUREAU_WEEK'] + df[
                            'AMT_REQ_CREDIT_BUREAU_MON'] + df['AMT_REQ_CREDIT_BUREAU_QRT'] + df['AMT_REQ_CREDIT_BUREAU_YEAR']
        df['ENQ_CREDIT_RATIO'] = df['AMT_ENQ_SUM'] / (df['AMT_CREDIT'] + 0.00001)
        
        # Binary
        for binary_feature in ["CODE_GENDER","FLAG_OWN_CAR", "FLAG_OWN_REALTY", "EMERGENCYSTATE_MODE"]:
            df[binary_feature], _ = pd.factorize(df[binary_feature])

        return df


    def neighbors_EXT_SOURCE_feature(self):
        '''
        Function to generate a feature which contains the means of TARGET of 500 neighbors of a particular row.
        '''        
        #https://www.kaggle.com/c/home-credit-default-risk/discussion/64821
            
        knn = KNeighborsClassifier(500, n_jobs = -1)
        
        train_data_for_neighbors = self.application_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','CREDIT_ANNUITY_RATIO']].fillna(0)
        train_target = self.application_train.TARGET
        test_data_for_neighbors = self.application_test[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','CREDIT_ANNUITY_RATIO']].fillna(0)
        
        knn.fit(train_data_for_neighbors, train_target)

        train_500_neighbors = knn.kneighbors(train_data_for_neighbors)[1]
        test_500_neighbors = knn.kneighbors(test_data_for_neighbors)[1]

        #adding the means of targets of 500 neighbors to new column
        self.application_train['TARGET_NEIGHBORS_500_MEAN'] = [self.application_train['TARGET'].iloc[ele].mean() for ele in train_500_neighbors]
        self.application_test['TARGET_NEIGHBORS_500_MEAN'] = [self.application_train['TARGET'].iloc[ele].mean() for ele in test_500_neighbors]


    def categories_based_feature_engineering(self, train_data, test_data) -> pd.DataFrame:
        """
        Performers grouping on various categorical columns and perform statistical aggregations.
        Then, joins this newly created features with the existed train or test dataset.
        
        Args:
            train_data ([type]): train dataset for categorical aggregations
            test_data ([type]): test dataset for categorical aggregations

        Returns:
            pd.DataFrame: Train and test datasets with new aggregations group on various cat variables.
        """
        columns_to_aggregate_on = [
            ['NAME_CONTRACT_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE'],
            ['CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_INCOME_TYPE'],
            ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE'],
            ['NAME_EDUCATION_TYPE','NAME_INCOME_TYPE','OCCUPATION_TYPE'],
            ['OCCUPATION_TYPE','ORGANIZATION_TYPE'],
            ['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY']

        ]
        aggregations = {
            'AMT_ANNUITY' : ['mean','max','min'],
            'ANNUITY_INCOME_RATIO' : ['mean','max','min'],
            'AGE_EMPLOYED_DIFF' : ['mean','min'],
            'AMT_INCOME_TOTAL' : ['mean','max','min'],
            'APARTMENTS_SUM_AVG' : ['mean','max','min'],
            'APARTMENTS_SUM_MEDI' : ['mean','max','min'],
            'EXT_SOURCE_MEAN' : ['mean','max','min'],
            'EXT_SOURCE_1' : ['mean','max','min'],
            'EXT_SOURCE_2' : ['mean','max','min'],
            'EXT_SOURCE_3' : ['mean','max','min']
        }
        
        for group in columns_to_aggregate_on:
            # grouping based on categories
            grouped_interactions = train_data.groupby(group).agg(aggregations) 
            grouped_interactions.columns = ['_'.join(ele).upper() + '_AGG_' + '_'.join(group) for ele in grouped_interactions.columns]
            # merging with the original data
            train_data = train_data.join(grouped_interactions, on = group)
            test_data = test_data.join(grouped_interactions, on = group)

        return train_data, test_data
        

    def main(self) -> pd.DataFrame:
        self.load_dataframe()
        self.clean_dataset()
        gc.collect()
        self.application_train = self.feature_engineering(self.application_train)
        self.application_test = self.feature_engineering(self.application_test)
        gc.collect()
        self.neighbors_EXT_SOURCE_feature()
        self.application_train, self.application_test = self.categories_based_feature_engineering(self.application_train, self.application_test)
        gc.collect()
        
        return self.application_train, self.application_test

In [18]:
application_train, application_test = preprocess_application_train_test().main()

In [19]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,EXT_SOURCE_MEAN_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_1_MEAN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_1_MAX_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_1_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_2_MEAN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_2_MAX_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_2_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_3_MEAN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_3_MAX_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_3_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY
0,100002,1,Cash loans,0,0,0,0,202500.0,406597.5,24700.5,...,0.057212,0.396730,0.931287,0.017177,0.490379,0.855,0.000006,0.503722,0.896010,0.000527
1,100003,0,Cash loans,1,0,1,0,270000.0,1293502.5,35698.5,...,0.023705,0.521413,0.946076,0.014568,0.510657,0.855,0.000010,0.503254,0.882530,0.000527
2,100004,0,Revolving loans,0,1,0,0,67500.0,135000.0,6750.0,...,0.070070,0.431679,0.936572,0.015053,0.525788,0.855,0.000005,0.512983,0.882530,0.000527
3,100006,0,Cash loans,1,0,0,0,135000.0,312682.5,29686.5,...,0.043409,0.556047,0.947649,0.017394,0.512272,0.855,0.000006,0.523489,0.893976,0.000527
4,100007,0,Cash loans,0,0,0,0,121500.0,513000.0,21865.5,...,0.057212,0.396730,0.931287,0.017177,0.490379,0.855,0.000006,0.503722,0.896010,0.000527


In [20]:
application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,EXT_SOURCE_MEAN_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_1_MEAN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_1_MAX_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_1_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_2_MEAN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_2_MAX_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_2_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_3_MEAN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_3_MAX_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY,EXT_SOURCE_3_MIN_AGG_CODE_GENDER_FLAG_OWN_CAR_FLAG_OWN_REALTY
0,100001,Cash loans,0,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0.057212,0.396730,0.931287,0.017177,0.490379,0.855000,0.000006,0.503722,0.896010,0.000527
1,100005,Cash loans,1,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0.043409,0.556047,0.947649,0.017394,0.512272,0.855000,0.000006,0.523489,0.893976,0.000527
2,100013,Cash loans,1,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0.065801,0.554838,0.962693,0.028066,0.531888,0.855000,0.000001,0.505731,0.887664,0.000527
3,100028,Cash loans,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0.057212,0.396730,0.931287,0.017177,0.490379,0.855000,0.000006,0.503722,0.896010,0.000527
4,100038,Cash loans,1,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0.068638,0.537381,0.941080,0.024761,0.534214,0.814758,0.000085,0.494432,0.881027,0.000527


## All tables merging

In [21]:
def merge_all_tables(
    application_train,
    application_test,
    bureau_aggregated,
    agg_previous_application,
    installments_payments,
    pos_aggregated,
    credit_card_balance) -> pd.DataFrame:
    """
    Function to merge all the tables together with the application_train and application_test tables
    on SK_ID_CURR.

    Inputs:
        All the previously pre-processed Tables.

    Returns:
        Single merged tables, one for training data and one for test data
    """
    app_train_merged = application_train.merge(
        bureau_aggregated, on="SK_ID_CURR", how="left"
    )
    app_test_merged = application_test.merge(
        bureau_aggregated, on="SK_ID_CURR", how="left"
    )

    app_train_merged = app_train_merged.merge(
        agg_previous_application, on="SK_ID_CURR", how="left"
    )
    app_test_merged = app_test_merged.merge(
        agg_previous_application, on="SK_ID_CURR", how="left"
    )

    app_train_merged = app_train_merged.merge(
        installments_payments, on="SK_ID_CURR", how="left"
    )
    app_test_merged = app_test_merged.merge(
        installments_payments, on="SK_ID_CURR", how="left"
    )

    app_train_merged = app_train_merged.merge(
        pos_aggregated, on="SK_ID_CURR", how="left"
    )
    app_test_merged = app_test_merged.merge(pos_aggregated, on="SK_ID_CURR", how="left")

    full_application_train = app_train_merged.merge(
        credit_card_balance, on="SK_ID_CURR", how="left"
    )
    full_application_test = app_test_merged.merge(
        credit_card_balance, on="SK_ID_CURR", how="left"
    )

    return full_application_train, full_application_test

In [22]:
train_data, test_data = merge_all_tables(
    application_train,
    application_test, 
    bureau_aggregated,
    agg_previous_application, 
    installments_payments,
    pos_aggregated,
    credit_card_balance)

In [23]:
print(
    f"After merging all of the datasets:\nTrain feature count: {train_data.shape[1]}\nTest feature count: {test_data.shape[1]}"
)

After merging all of the datasets:
Train feature count: 1120
Test feature count: 1119


## All tables cleaning

### Description
1. Drop missing values above or equal to 80% threshold.
2. Encode categorical variables using OneHotEncoder thus unknown categories could be handled appropriately during the testing phase.
3. Drop correlated features when pearson coefficient is equal or more than 0.9
4. Perform cross validated features selection based on LGBMClassifier. This part has been added after initial modelling has been done and extraordinary dimensionality was too computationally expensive.

### Final table cleaning

In [24]:
class process_agg_application_features:
    """
    Final processing of fully merged Home Depot datasets.
    Drops missing values for certain threshold, encodes categorical variables.
    Then, drops highly correlated features, and finally performs feature selection.
    """

    def __init__(self, train_data: pd.DataFrame, test_data: pd.DataFrame):
        self.train_data = train_data
        self.test_data = test_data

    def drop_missing_values(self):
        """
        Drops columns with missing values equal or more than 80% of total.
        """
        missing_values_as_perc = (
            self.train_data.isnull().sum() / len(self.train_data)
        ).sort_values(ascending=False)
        columns_missing_more_than_80_perc = missing_values_as_perc.index[
            missing_values_as_perc > 0.8
        ]
        self.train_data.drop(
            columns=columns_missing_more_than_80_perc, axis=1, inplace=True
        )
        self.test_data.drop(
            columns=columns_missing_more_than_80_perc, axis=1, inplace=True
        )
        gc.collect()

    def encode_categoricals(self):
        """
        Encode categorical variables using OneHotEncoder.
        """
        encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
        categorical_cols = self.train_data.select_dtypes(exclude="number").columns
        enc_train_cols = pd.DataFrame(
            encoder.fit_transform(self.train_data[categorical_cols])
        )
        enc_test_cols = pd.DataFrame(
            encoder.transform(self.test_data[categorical_cols])
        )

        enc_train_cols.index = self.train_data.index
        enc_test_cols.index = self.test_data.index

        num_X = self.train_data.drop(categorical_cols, axis=1)
        num_X_test = self.test_data.drop(categorical_cols, axis=1)

        self.train_data = pd.concat([num_X, enc_train_cols], axis=1)
        self.test_data = pd.concat([num_X_test, enc_test_cols], axis=1)
        gc.collect()

    def drop_correlated_features(self):
        """
        Drop feature columns with a correlation higher than 0.9 (Pearson's coefficient)
        """
        threshold = 0.9
        corr_matrix = self.train_data.corr().abs()
        upper = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_)
        )
        correlated_to_drop = [
            column for column in upper.columns if any(upper[column] > threshold)
        ]
        self.train_data.drop(columns=correlated_to_drop, axis=1, inplace=True)
        self.test_data.drop(columns=correlated_to_drop, axis=1, inplace=True)
        gc.collect()

    def drop_zero_importance_features(self):
        """
        Perform cross validated feature selection based on LGBMClassifier.
        Then, filter out non incremental features and keep only non-zero important ones.
        """
        self.important_columns = set()
        score = 1
        y = self.train_data.pop("TARGET")

        while score > 0.72:
            selection_data = self.train_data.drop(list(self.important_columns), axis=1)
            num_folds = 3
            fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)
            score = 0
            model_feature_importance = np.zeros_like(selection_data.columns)

            for fold_num, (train_indices, val_indices) in enumerate(
                fold.split(selection_data, y), 1
            ):
                x_train = selection_data.iloc[train_indices]
                x_val = selection_data.iloc[val_indices]
                y_train = y.iloc[train_indices]
                y_val = y.iloc[val_indices]

                lg = LGBMClassifier(n_jobs=-1, random_state=1)
                lg.fit(x_train, y_train)

                model_feature_importance += lg.feature_importances_ / num_folds
                score += roc_auc_score(y_val, lg.predict_proba(x_val)[:, 1]) / num_folds

            imp_cols_indices = np.where(np.abs(model_feature_importance) > 0)
            cols_imp = self.train_data.columns[imp_cols_indices]

            if score > 0.7:
                self.important_columns.update(cols_imp)
            gc.collect()

        self.important_columns = list(self.important_columns)
        self.train_data = self.train_data[self.important_columns]
        self.train_data = pd.concat([self.train_data, y], axis=1)
        self.test_data = self.test_data[self.important_columns]
        gc.collect()

    def main(self) -> pd.DataFrame:
        self.drop_missing_values()
        self.encode_categoricals()
        self.drop_correlated_features()
        self.drop_zero_importance_features()

        return self.train_data, self.test_data

In [25]:
process_features = process_agg_application_features(train_data, test_data)
train_data, test_data = process_features.main()

In [26]:
print(
    f"After merging all of the datasets:\nTrain feature count: {train_data.shape[1]}\nTest feature count: {test_data.shape[1]}"
)

After merging all of the datasets:
Train feature count: 526
Test feature count: 525


In [27]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)